In [8]:
from totalface.model_zoo.model_common import load_onnx,load_tensorRT,load_openvino
from totalface.data import read_image

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [4]:
def normalization(rgb_img,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    MEAN = 255 * np.array(mean_list)
    STD = 255 * np.array(std_list)
    rgb_img = rgb_img.transpose(-1, 0, 1)
    norm_img = (rgb_img - MEAN[:, None, None]) / STD[:, None, None]
    
    return norm_img

In [35]:
# onnx (norm, numpy array, (H,W,C))
lb_num=5
input_size=112
model_type='onnx'
model_path = "./dan_230503_e34.onnx"
model = load_onnx.Onnx_session(model_path,input_mean=0.0, input_std=1.0,output_sort=True,onnx_device='cuda')

providers: ['CUDAExecutionProvider']


In [ ]:
# tensorRT (norm, torch tensor, (N,C,H,W)
lb_num=5
input_size=112
model_type='trt'
model_path = "dan_230503_e34.v8.trt"
model = load_tensorRT.TrtModel(model_path,torch_image=True,not_norm=True)

In [46]:
# openvino (norm, torch tensor, (N,C,H,W)
lb_num=5
input_size=112
model_type='openvino'
model_path = ["dan_230503_e34.xml", \
              "dan_230503_e34.bin"]
model = load_openvino.Openvino(model_path,not_norm=True,torch_image=True)

In [47]:
if lb_num==5:
    exp_cls = ['happy','surprise','anger','sorrow','neurality']
elif lb_num==7:
    exp_cls = ["happy","embarrassed","anger","anxious","hurt","sorrow","neutrality"]

In [48]:
import torch

In [49]:
img_path = "./test_aimg.jpg" # aligned image
img = read_image(img_path)
img = cv2.resize(img,(input_size,input_size))
norm_aimg = normalization(img)
if model_type=='onnx':
    norm_aimg = norm_aimg.transpose(1,2,0) # (3,size,size) -> (size,size,3) 
else:
    norm_aimg = torch.from_numpy(norm_aimg).unsqueeze(0)

In [50]:
_,_,out = model(norm_aimg)

In [54]:
pred = np.argmax(out)
pred = exp_cls[pred]
print(pred)

surprise


In [41]:
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [52]:
output_sf = softmax(out)[0]

In [53]:
output_sf

array([2.3142078e-04, 6.8300980e-01, 1.0039210e-03, 3.1351376e-01,
       1.7487615e-03, 2.3811695e-04, 2.5419690e-04], dtype=float32)